# Mobile games development

## Задание

##### Вам предстоит поработать аналитиком данных в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:
##### 1) Расчётать Retention (по дням от даты регистрации);
##### 2) На основе данных, полученных в ходе проведения A/B теста, определить, какой набор акционных предоложений можно считать лучшим и на основе каких метрик стоит принять правильное решение;
##### 3) Предложить метрики для оценки результатов последнего проведенного тематического события в игре.

## Данные

##### Расчёр Retention
##### shared/problem1-reg_data.csv – данные о времени регистрации (таблица из 2 колонок):
##### 1) reg_ts - время регистрации в формате Unix;
##### 2) uid - id пользователя;
##### shared/problem1-auth_data.csv – данные о времени захода пользователей в игру (таблица из 2 колонок):
##### 1) auth_ts - время повторного входа в игру в формате Unix;
##### 2) uid - id пользователя.
##### Определение набора акционных предложений
##### Проект_1_Задание_2.csv - данные о выручке и группах (таблица из 3 колонок):
##### 1) user_id - id пользователя;
##### 2) revenue - выручка;
##### 3) testgroup - вид группы (a - контрольная, b - тестовая).

In [5]:
import pandas as pd

task1_reg = pd.read_csv('reg_data.csv', sep=';')
task1_auth = pd.read_csv('auth_data.csv', sep=';')

### Расчет Retention

In [6]:
def retention_by_day(reg_data, auth_data):
    
    reg_data.reg_ts = pd.to_datetime(reg_data.reg_ts, unit='s').dt.date
    auth_data.auth_ts = pd.to_datetime(auth_data.auth_ts, unit='s').dt.date
    
    merged_data = pd.merge(reg_data, auth_data, on='uid', how='left')
    merged_data['diff'] = (merged_data.auth_ts - merged_data.reg_ts).dt.days
     
    retention_data = merged_data.groupby(['reg_ts', 'diff'], as_index=False).\
                                 agg(retained=('uid', 'nunique'))
    
    reg_by_day = task1_reg.groupby('reg_ts', as_index=False).\
                           agg(cnt=('uid', 'nunique'))

    retention_data = pd.merge(retention_data, reg_by_day, on='reg_ts')
    retention_data['ret_rate'] = retention_data.retained / retention_data.cnt
    

    return retention_data



retention_by_day(task1_reg, task1_auth)

,reg_ts,diff,retained,cnt,ret_rate
0,1998-11-18,0,1,1,1.000000
1,1999-07-22,0,1,1,1.000000
2,1999-07-22,3,1,1,1.000000
3,1999-07-22,9,1,1,1.000000
4,1999-07-22,14,1,1,1.000000
...,...,...,...,...,...
2716223,2020-09-21,1,31,1638,0.018926
2716224,2020-09-21,2,49,1638,0.029915
2716225,2020-09-22,0,1641,1641,1.000000
2716226,2020-09-22,1,14,1641,0.008531


### Определение набора акционных предложений
##### Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.
##### Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

In [4]:
task2 = pd.read_csv('sets_shares.csv', sep=';')

In [74]:
# Конверсия в платящего пользователя (CR) - процент платящих пользователей
# CR = количество платящих пользователей / общее количество пользователей

control_cr = 1928 / 202103
test_cr = 1805 / 202667

control_cr > test_cr # True, следовательно акционные предложения в тестовой группе менее привлекательны

# Средниц доход на платящего пользователя (ARPPU)
# ARPPU = общий доход / количество платящих пользователей

# Разбиваю датафрейм на 2 (данные о контрольной и тестовой группах)
control = task2.query('testgroup=="a"')
test = task2.query('testgroup=="b"')

# Нахожу общие доходы групу
ctrl_com_rev = sum(control.revenue)
tst_com_rev = sum(test.revenue)

# Считаю количество платящих пользователей каждой группы
ctrl_pay_users = control.query('revenue!=0').\
                         revenue.count()
test_pay_users = test.query('revenue!=0').\
                      revenue.count()

# Нахожу ARPPU каждой группы
ctrl_arppu = ctrl_com_rev / ctrl_pay_users # 2663.9984439834025
test_arppu = tst_com_rev / test_pay_users # 3003.6581717451522

diff = ctrl_arppu - test_arppu # 339.65972776174976

# Разница сильно велика, поэтому делаю вывод о том, что тестовый набор предложений лучше

### Метрики для оценки события
##### В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?
##### Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

In [9]:
# Метрики для обычного события

# Количество новых игроков во время проведения события
# Среднее время игрового сеасна игроков
# Средний прогресс игроков в событии (уровень после завершения события)
# Количество игроков, которые получили все награды, прошли событие до конца
# ARPU
# ARPPU
# Конверсия в платящего пользователя
# Retention


# Метрики для события с усложненной механикой (остаются прошлые метрики и добавляются новые):

# Среднее количетсво откатов игроков
# Среднее количество уровней, на которые откатились игроки
# Уровень фрустрации (процент оттока после отката уровней)
# ARPPU откатившихся игроков